# NDVI anomalies

The first step is to load all the required libraries

In [ ]:
import folium
import folium.plugins
import geopandas as gpd
import shapely.geometry
from IPython.display import HTML, display
from pystac_client import Client
from odc.stac import configure_rio, stac_load
import pathlib
import json
import rasterio
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import zarr
import rioxarray
import os

Now we need to configure and start a session in the datacube, to do this, we need to log-in using a token

In [ ]:
root = pathlib.Path().resolve().parent.parent
root

In [ ]:
TOKEN_PATH = "path to token"
HDC_STAC_URL= "url"

import os
def _get_hdc_stac_param_from_env():
    
    if "JUPYTERHUB_USER" in os.environ:
    
        signer = None
        header = None
        aws={}   # Get credentials for accessing S3 bucket 

    else:

        def make_signer(fname="./tk.json"):
            """
            Loads token from file at fname, and returns a function patching request urls with said token
            """
            tk = ""
            with open(fname, "rt") as src:
                tk = json.load(src)["tk"]

            def sign(url, _tk=tk):
                signed = f"{url}?{_tk}"
                return signed

            return sign

        signer = make_signer(TOKEN_PATH)
        header = {"origin": "https://wfp.org"}
        aws = None
        
    # Instantiate an API client pointing to the WFP HDC STAC API
    hdc_stac_client = Client.open(HDC_STAC_URL, headers=header)
    
    # Set up GDAL/rasterio configuration.
    configure_rio(cloud_defaults=True, verbose=True, aws=aws)
        
    return hdc_stac_client, signer


#
# STAC CLIENTS
#

hdc_stac_client, signer = _get_hdc_stac_param_from_env() 

In [ ]:
# Ask the user to select an option
print("Please select the pilot area:")
print("1. COL")
print("2. CHAD")
print("3. IRAQ Dahuk")
print("4. IRAQ Najaf")
pilot = input()
assert pilot in ['1', '2', '3', '4'], "Invalid pilot area selected."

In [ ]:
if pilot == "1":
    #input_shp = "C:/Geotar/COL/geodata/Processed/Education/Education_facilities.shp"
    pilot_name = "COL"
    mask_shp = "C:/Geotar/COL/geodata/Processed/Mask/COL_mask.shp"
    period = "2021-05-01/2022-01-31"
    #output = f"C:/Geotar/COL/geodata/Processed/{res_folder}/dist_"+ out_name
    print("You selected Colombia")
elif pilot == "2":
    pilot_name = "CHAD"
    #input_shp = "zip://C:/Geotar/CHAD/geodata/Processed/Education/hotosm_chad_education_facilities_points_shp.zip/hotosm_chad_education_facilities_points.shp"
    mask_shp = "C:/Geotar/CHAD/geodata/Processed/Mask/Chad_mask.shp"
    period = "2022-05-01/2023-01-31"
    #periodlta = "1970-01-01/1970-12-31"
    #output = f"C:/Geotar/CHAD/geodata/Processed/{res_folder}/dist_{res_folder}_"+ out_name
    print("You selected CHAD")
elif pilot == "3":
    pilot_name = "IRAQ_D"
    #input_shp = "zip://C:/Geotar/IRAQ_D/geodata/Raw/Education/hotosm_irq_education_facilities_points_shp.zip/hotosm_irq_education_facilities_points.shp"
    mask_shp = "C:/Geotar/IRAQ_D/geodata/Processed/Mask/Dahuk_mask.shp"
    period = "2021-11-01/2022-05-31"
    #output = f"C:/Geotar/IRAQ_D/geodata/Processed/{res_folder}/dist_"+ out_name
    print("You selected IRAQ Dahuk")
elif pilot == "4":
    pilot_name = "IRAQ_N"
    period = "2021-11-01/2022-05-31"
    #input_shp = "zip://C:/Geotar/IRAQ_N/geodata/Raw/Education/hotosm_irq_education_facilities_points_shp.zip/hotosm_irq_education_facilities_points.shp"
    mask_shp = "C:/Geotar/IRAQ_N/geodata/Processed/Mask/Najaf_mask.shp"
    #output = f"C:/Geotar/IRAQ_N/geodata/Processed/{res_folder}/dist_"+ out_name
    print("You selected IRAQ Najaf")

The next step is to get the NDVI data from the data cube, to do so, we need to define the bounding box of the pilot area of interest

In [ ]:
# Load the shapefiles
area_shp = gpd.read_file(mask_shp)

# Get the bounding box of the shapefile
bbox = area_shp.total_bounds
bbox

View the location of the bounding box 

In [ ]:
# Create a map centered at the center point of the polygon
m = folium.Map(location=[bbox[1], bbox[0]], zoom_start=7)

# Add a polygon to the map
folium.Polygon(
    [
        [bbox[1], bbox[0]],
        [bbox[1], bbox[2]],
        [bbox[3], bbox[2]],
        [bbox[3], bbox[0]],
        [bbox[1], bbox[0]],
    ],
    color='red',
    fill_color='red',
    fill_opacity=0.2
).add_to(m)

# Display the map
m

Fetch the dekadal NDVI anomaly data from HDC

In [ ]:
#lt_dates = "2002-07-01/2018-07-01"
query_ndvi_lta = hdc_stac_client.search(bbox=bbox,
    #collections=["mod13q1_vim_native"],
    collections=["mxd13q1_viq_dekad"], #mxd13a2_vim_dekad_lta
    datetime= period).get_all_items()#1970-01-01T00:00:00Z/1970-12-31T00:00:00Z
res = 0.0022457882102988 # 250 or 0.01 for 1km
ndvi_lta = stac_load(query_ndvi_lta, patch_url=signer, output_crs='EPSG:4326', resolution= res, bbox=bbox)
ndvi_lta

Save the ndvi anomaly zarr file

In [ ]:
# create a directory to store the geotiff files
output_dir_zarr = f"C:/Geotar/{pilot_name}/geodata/zarr"
if not os.path.exists(output_dir_zarr):
    os.makedirs(output_dir_zarr)
outfile = output_dir_zarr + "/ndvi_an_stac.zarr"
ndvi_lta.to_zarr(outfile)
print(f"{outfile} saved")

Group the data by month

In [ ]:
m_ndvi_anom = ndvi_lta.groupby('time.month').mean('time')
m_ndvi_lta_r = m_ndvi_anom #/100  this rescaling is not working
m_ndvi_lta_r

In [ ]:
s_ndvi_anom = m_ndvi_anom.mean(dim=["month"])
s_ndvi_anom
image = s_ndvi_anom/100 # Rescaling applied here
output_dir_s = f"C:/Geotar/{pilot_name}/geodata/Processed/Vegetation/season"
filename_s = f'{output_dir_s}/ndvi_a_m.tif'
if not os.path.exists(output_dir_s):
    os.makedirs(output_dir_s)

# write the data to a geotiff file
image.rio.to_raster(filename_s, driver='GTiff')
print(f"{filename_s} saved successfully")

In [ ]:
s_ndvi_anom_max = m_ndvi_anom.max(dim=["month"])
s_ndvi_anom_max
image_max = s_ndvi_anom_max/100 # Rescaling applied here
output_dir_s = f"C:/Geotar/{pilot_name}/geodata/Processed/Vegetation/season"
filename_s_max = f'{output_dir_s}/ndvi_a_ma.tif'
if not os.path.exists(output_dir_s):
    os.makedirs(output_dir_s)

# write the data to a geotiff file
image_max.rio.to_raster(filename_s_max, driver='GTiff')
print(f"{filename_s_max} saved successfully")

Exports a geotiff file for each date

In [ ]:
# create a directory to store the geotiff files
output_dir = f"C:/Geotar/{pilot_name}/geodata/Processed/Vegetation"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# loop over all timesteps in the dataset
for i in range(len(m_ndvi_lta_r.month)):
    # extract a single timestep as a DataArray
    image = m_ndvi_lta_r.isel(month=i)/100 # Rescaling applied here
    # create a file path for the geotiff file
    filename = f'{output_dir}/ndvi_a_{m_ndvi_lta_r.month.values[i]}.tif'

    # write the data to a geotiff file
    image.rio.to_raster(filename, driver='GTiff')
    print(f"{filename} saved successfully")

### Fetch NDVI dekadal data for the period and pilot of interest

In [ ]:
%%time
NDVI = hdc_stac_client.search(bbox=bbox,
    #collections=["mod13q1_vim_native"],
    collections=["mxd13q1_vim_dekad"],
    datetime=period, #emulates the period of data cube files
).get_all_items()

res = 0.0022457882102988 # 250 or 0.01 for 1km
ndvi_stack = stac_load(NDVI, output_crs='EPSG:4326', resolution= res , patch_url=signer, bbox=bbox)
#
#ndvi_stack = stac_load(NDVI,  patch_url=signer, bbox=bbox)
ndvi_stack

In [ ]:
# create a directory to store the geotiff files
output_dir_zarr = f"C:/Geotar/{pilot_name}/geodata/zarr"
if not os.path.exists(output_dir_zarr):
    os.makedirs(output_dir_zarr)
outfile_zarr = output_dir_zarr + "/ndvi_stac.zarr"
print(f'path to zarr file: {outfile_zarr}')

save the zarr file to disk

In [ ]:
# save zarr file
ndvi_stack.to_zarr(outfile_zarr)
print(f"{outfile_zarr} saved")

In [ ]:
#Load zarr file containing NDVI dekadal data for the season
ndvi_stack = xr.open_zarr(outfile_zarr)
ndvi_stack

Aggregate the dekadal NDVI by month

In [ ]:
#m_ndvi = ndvi_stack.resample(time='1M').mean('time')
m_ndvi = ndvi_stack.groupby('time.month').mean('time')
m_ndvi = m_ndvi * 0.0001
m_ndvi

In [ ]:
#Aggregate the data by season
ndvi_m_s = m_ndvi.mean(dim=["month"])
ndvi_m_s

output_dir_s = f"C:/Geotar/{pilot_name}/geodata/Processed/vegetation/season"
filename_m_s = f'{output_dir_s}/ndvi_m_s.tif'
if not os.path.exists(output_dir_s):
    os.makedirs(output_dir_s)

# write the data to a geotiff file
ndvi_m_s.rio.to_raster(filename_m_s, driver='GTiff')
print(f"{filename_m_s} saved successfully")

save the monthly ndvi files

In [ ]:
# create a directory to store the geotiff files
output_dir = f"C:/Geotar/{pilot_name}/geodata/Processed/Vegetation"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# loop over all timesteps in the dataset
for i in range(len(m_ndvi.month)):
    # extract a single timestep as a DataArray
    image_ndvi = m_ndvi.isel(month=i)
    # create a file path for the geotiff file
    filename = f'{output_dir}/ndvi_{m_ndvi.month.values[i]}.tif'

    # write the data to a geotiff file
    image_ndvi.rio.to_raster(filename, driver='GTiff')
    print(f"{filename} saved successfully")
